In [14]:
import pandas as pd
import numpy as np
import pickle #피클로 저장할 때 불러오는 모듈
from konlpy.tag import * #형태소 분석기(kkma, okt 등) 사용
from sklearn.linear_model import LogisticRegression #학습모델
from sklearn.pipeline import Pipeline #학습모델과 tfidf를 한곳에 넣는 파이프라인 즉 두개의 모델을 하나로사용
from sklearn.model_selection import GridSearchCV #하이퍼파라미터튜닝
from sklearn.feature_extraction.text import TfidfVectorizer #형태소로 분류된 단어에 가중치 추가

In [15]:

    # 수집한 데이터를 읽어온다.
df = pd.read_csv('./data/블루밍데이터셋.csv',encoding='CP949')

    # 전처리를 수행한다.
X_p = df['term']
y_p = df['label']

    # 학습데이터와 테스트 데이터로 나눈다.
#X_p = df['term'].tolist()
#y_p = df['label'].tolist()

from sklearn.model_selection import train_test_split

    # 70%는 학습, 30%는 test
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size=0.3, random_state=0)


In [16]:
# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.pos(text,norm=True,stem=True)

In [17]:
# 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer,analyzer='char',ngram_range = (1, 2),sublinear_tf = True)

# 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

# tfidf와 logistic을 하나의 파이프에 저장
pipe_p = Pipeline([('vect', tfidf), ('clf', logistic)])

# 학습한다.
pipe_p.fit(X_train_p, y_train_p)

# 학습 정확도 측정
y_pred = pipe_p.predict(X_test_p)



C:\dev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
#하이퍼파라미터 튜닝
grid_params_p = {
    'vect__min_df' : [3,5,10],
    'vect__max_df' : [20000,22000,24000],
    'vect__ngram_range' : [(1,2),(1,3),(1,4)],
    'clf__C' : [0.001,0.01,0.1,1,10,100]
}

In [19]:
grid_pipe_p = GridSearchCV(pipe_p,grid_params_p,cv=5,n_jobs=-1) #cv=5는 5번 교차검증 하겠다

In [20]:
grid_pipe_p.fit(X_train_p,y_train_p) #그리드 학습

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(analyzer='char',
                                                        lowercase=False,
                                                        ngram_range=(1, 2),
                                                        sublinear_tf=True,
                                                        tokenizer=<function tokenizer at 0x0000022CDAC8ED30>)),
                                       ('clf',
                                        LogisticRegression(C=10.0,
                                                           random_state=0))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'vect__max_df': [20000, 22000, 24000],
                         'vect__min_df': [3, 5, 10],
                         'vect__ngram_range': [(1, 2), (1, 3), (1, 4)]})

In [21]:
grid_pipe_p.best_params_ #최고의 파라미터값 추출

{'clf__C': 10,
 'vect__max_df': 20000,
 'vect__min_df': 3,
 'vect__ngram_range': (1, 4)}

In [22]:
best_model_pipe_p = grid_pipe_p.best_estimator_ #최고의 파라미터로 적용시킨후 정확도 추출
best_model_pipe_p.score(X_test_p,y_test_p)

0.9309964726631393

In [23]:
ff = ['나는 불행하다']

In [24]:
best_model_pipe_p.predict(ff)[0] #테스트 해보기

-1

In [25]:
#웹 페이지와 연동시키기 위한 플라스크 서버 구축

from flask import Flask
from flask import request, redirect

#app 객체 생성
app = Flask(__name__) 

#사용자가 일기 작성후 확인 누르면 먼저 여기로 들어와서 내용 분석후 자바로 보내서 데이터베이스에 저장
@app.route('/DiaryPost1', methods=['GET','POST']) 
def index():     
       #get 방식으로 받으면 args로
    if request.method == 'GET':
        DI_CONTENTS = request.args['DI_CONTENTS']
        DI_TITLE = request.args['DI_TITLE']
        DI_DATE = request.args['DI_DATE']
        #post 방식으로 받으면 form으로
    elif request.method == 'POST':
        DI_CONTENTS = request.form['DI_CONTENTS']
        DI_TITLE = request.form['DI_TITLE']
        DI_DATE = request.form['DI_DATE']
        
    #값이 잘 넘어오고 분석 잘되는지 print로 테스트해보기
    a = best_model_pipe_p.predict([DI_CONTENTS])[0]
    print(a)
    print(DI_CONTENTS)
    print(DI_TITLE)
    print(DI_DATE)
    #웹페이지 주소와 쿼리스트링 기술로 값 보내주기
    url = 'http://localhost:8085/Blooming/DiaryPost?DI_SCORE={}&DI_TITLE={}&DI_DATE={}&DI_CONTENTS={}'.format(best_model_pipe_p.predict([DI_CONTENTS])[0],DI_TITLE,DI_DATE,DI_CONTENTS)
    return redirect(url,code=307) #code=307 은 post방식으로 보내주는 상태코드


if __name__ == '__main__':
    app.run(host='172.20.10.3', port='5000')  #ip주소와 포트번호 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.20.10.3:5000/ (Press CTRL+C to quit)
172.20.10.4 - - [26/Nov/2020 10:47:40] "POST /DiaryPost1 HTTP/1.1" 307 -


1
최종발표 잘할 것 같다.
최종발표
2020-11-29


172.20.10.4 - - [26/Nov/2020 11:37:26] "POST /DiaryPost1 HTTP/1.1" 307 -


-1

기분이좋다
2020-11-26


...

1
행복하다
사랑
2020-12-30


172.20.10.4 - - [26/Nov/2020 13:28:17] "POST /DiaryPost1 HTTP/1.1" 307 -


1
행복해
일기
2020-12-31


172.20.10.4 - - [26/Nov/2020 13:29:01] "POST /DiaryPost1 HTTP/1.1" 307 -


1
오늘은 즐거운 목요일
안녕
2021-01-01
